In [1]:
import cv2 as cv
import numpy as np

In [19]:
def detect_points(gray):
    p = cv.goodFeaturesToTrack(
        gray,
        maxCorners=500,
        qualityLevel=0.01,
        minDistance=7,
        blockSize=7)
    if p is None:
        return None
    return p.astype(np.float32)

In [15]:
def main():
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Cannot open webcam")

    ret, old_frame = cap.read()
    if not ret:
        raise RuntimeError("Cannot read first frame")

    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    p0 = detect_points(old_gray)
    
    lk_params = dict(
        winSize=(21, 21),
        maxLevel=3,
        criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01),
        flags=0,
        minEigThreshold=1e-4)

    # Optional: forward-backward consistency check threshold (pixels)
    fb_thresh = 1.5

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        vis = frame.copy()

        if p0 is not None and len(p0) > 0:
            p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

            # Forward-backward check (more robust filtering)
            p0_back, st_back, _ = cv.calcOpticalFlowPyrLK(gray, old_gray, p1, None, **lk_params)

            st = st.reshape(-1)
            st_back = st_back.reshape(-1)
            p0r = p0.reshape(-1, 2)
            p1r = p1.reshape(-1, 2)
            p0br = p0_back.reshape(-1, 2)

            fb_err = np.linalg.norm(p0r - p0br, axis=1)

            good = (st == 1) & (st_back == 1) & (fb_err < fb_thresh)

            good_old = p0r[good]
            good_new = p1r[good]

            for (x0, y0), (x1, y1) in zip(good_old, good_new):
                cv.line(vis, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
                cv.circle(vis, (int(x1), int(y1)), 2, (0, 0, 255), -1)

            p0 = good_new.reshape(-1, 1, 2).astype(np.float32)

        # Re-detect if too few points remain, or periodically
        if p0 is None or len(p0) < 80 or (frame_idx % 30 == 0):
            new_p = detect_points(gray)
            if new_p is not None:
                p0 = new_p

        cv.putText(vis, "ESC: quit | r: re-detect", (10, 25),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv.imshow("Day 10: Sparse Optical Flow (PyrLK)", vis)

        key = cv.waitKey(1) & 0xFF
        if key == 27:
            break
        if key == ord('r'):
            p0 = detect_points(gray)

        old_gray = gray
        frame_idx += 1

    cap.release()
    cv.destroyAllWindows()

In [18]:
main()